In [14]:
import pandas as pd

In [15]:
df = pd.read_csv('../popular_songs_metadata.csv')

In [20]:
CLIENT_ID = 'c69dcc8b88574693bb9a8f67269450b3'
CLIENT_SECRET = 'a2b4ab0e224e4863aeab704bd92fdc08'
REDIRECT_URI = 'https://localhost:8080/callback/'

In [21]:
import base64
import requests

def get_access_token(client_id, client_secret):
    """Obtain the access token from Spotify."""
    endpoint = "https://accounts.spotify.com/api/token"
    encoded_credentials = base64.b64encode(f"{client_id}:{client_secret}".encode()).decode()
    headers = {
        "Authorization": f"Basic {encoded_credentials}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    payload = {
        "grant_type": "client_credentials"
    }

    response = requests.post(endpoint, headers=headers, data=payload)
    response_data = response.json()
    
    if response.status_code == 200:
        return response_data["access_token"]
    else:
        raise Exception("Failed to obtain access token")


def search_track(access_token, track_name, artist_name):
    """Search for a track on Spotify and return its first result."""
    query = f"track:{track_name} artist:{artist_name}"
    endpoint = f"https://api.spotify.com/v1/search?q={query}&type=track&limit=1"
    headers = {"Authorization": f"Bearer {access_token}"}
    
    response = requests.get(endpoint, headers=headers)
    results = response.json()
    
    if results['tracks']['items']:
        return results['tracks']['items'][0]
    else:
        return None
    
def get_artist_genres(access_token, artist_id):
    """Get genres for a given artist."""
    endpoint = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    
    response = requests.get(endpoint, headers=headers)
    artist_info = response.json()
    
    return artist_info.get('genres', [])


def get_audio_features(access_token, track_id):
    """Get audio features for a given track."""
    endpoint = f"https://api.spotify.com/v1/audio-features/{track_id}"
    headers = {"Authorization": f"Bearer {access_token}"}
    
    response = requests.get(endpoint, headers=headers)
    audio_features = response.json()
    
    # Depending on your needs, you might want to return only specific features
    return audio_features

def update_df_with_spotify_data(df, access_token):
    """Update DataFrame rows with Spotify data, including audio features and artist genres.
    Saves the DataFrame to a CSV file every 1000 lines."""
    for index, row in df.iterrows():
        track_name = row['title']
        artist_name = row['artist_name']

        track_info = search_track(access_token, track_name, artist_name)
        if track_info:
            artist_id = track_info['artists'][0]['id']
            track_id = track_info['id']  # Get the track's ID for audio features
            genres = get_artist_genres(access_token, artist_id)
            
            # Save genres as a comma-separated string
            df.at[index, 'genres'] = ', '.join(genres)
            
            audio_features = get_audio_features(access_token, track_id)
            # Save requested audio features
            df.at[index, 'key'] = audio_features.get('key')
            df.at[index, 'loudness'] = audio_features.get('loudness')
            df.at[index, 'mode'] = audio_features.get('mode')
            df.at[index, 'speechiness'] = audio_features.get('speechiness')
            df.at[index, 'acousticness'] = audio_features.get('acousticness')
            df.at[index, 'instrumentalness'] = audio_features.get('instrumentalness')
            df.at[index, 'liveness'] = audio_features.get('liveness')
            df.at[index, 'valence'] = audio_features.get('valence')
            df.at[index, 'tempo'] = audio_features.get('tempo')
            
            print(f"Updated row {index + 1} with genres and audio features for '{track_name}' by '{artist_name}'.")

                # Save the DataFrame to a CSV file every 1000 lines
            if (index + 1) % 1000 == 0:
                backup_filename = "spotify_data_api.csv"
                df.to_csv(backup_filename, index=False)
                print(f"DataFrame saved to {backup_filename} after {index + 1} rows.")
        else:
            print(f"Track '{track_name}' by '{artist_name}' not found. Skipping.")

    # Final save to ensure any remaining changes are also persisted
    final_filename = "spotify_data_final_2.csv"
    df.to_csv(final_filename, index=False)
    print(f"Final DataFrame saved to {final_filename}.")

    return df


In [22]:
access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)   
print(access_token)

BQAkmesB3NQWOZa90t8yavdaECqZZJ4cYHaO3vw54GIy_CY0jo4ApAeOEpVa7eKiKEvn7HXT5pql2v7oiSUj8QTSKTfQ13VPqZ1j-K6vO1NYc8dbb_E


In [23]:


# First, obtain an access token
#access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

subset_df = df.iloc[6079:]


updated_df = update_df_with_spotify_data(subset_df, access_token)




/var/folders/sw/6q2vf2dx0r502xm9jxzvdxzc0000gn/T/ipykernel_29737/3667872320.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'genres'] = ', '.join(genres)
/var/folders/sw/6q2vf2dx0r502xm9jxzvdxzc0000gn/T/ipykernel_29737/3667872320.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.at[index, 'key'] = audio_features.get('key')
/var/folders/sw/6q2vf2dx0r502xm9jxzvdxzc0000gn/T/ipykernel_29737/3667872320.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a

Updated row 6080 with genres and audio features for 'I Am Not Ashamed' by 'The Brooklyn Tabernacle Choir'.
Updated row 6081 with genres and audio features for 'Si La Vie M'A Mis Là' by 'Tryo'.
Updated row 6082 with genres and audio features for 'Si La Vie M'A Mis Là' by 'Tryo'.
Updated row 6083 with genres and audio features for 'Si La Vie M'A Mis Là' by 'Tryo'.
Updated row 6084 with genres and audio features for 'Si La Vie M'A Mis Là' by 'Tryo'.
Updated row 6085 with genres and audio features for 'Si La Vie M'A Mis Là' by 'Tryo'.
Updated row 6086 with genres and audio features for 'Dona Cila' by 'Maria Gadú'.
Updated row 6087 with genres and audio features for 'Dona Cila' by 'Maria Gadú'.
Updated row 6088 with genres and audio features for 'Dona Cila' by 'Maria Gadú'.
Updated row 6089 with genres and audio features for 'Dona Cila' by 'Maria Gadú'.
Updated row 6090 with genres and audio features for 'Janela' by 'Babado Novo'.
Track 'No Problem (Explicit Version)' by 'Lil Scrappy' not f

ConnectTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Max retries exceeded with url: /v1/search?q=track:Cold%20Blooded%20(Acid%20Cleanse)%20artist:The%20fFormula&type=track&limit=1 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x1490c4bf0>, 'Connection to api.spotify.com timed out. (connect timeout=None)'))

In [33]:
df_extrainfo = pd.read_csv('./spotify_data_api.csv')
df_extrainfo.head()

,user,song,title,play_count,release,artist_name,year,genres,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,b493eda768d25cfb48dec2567b0826cdb9f6eef4,SOCBSZW12AB01891C1,XRDS,256,Ritual Noise,Covenant,0,"aggrotech, cyberpunk, dark wave, ebm, electro-...",5.0,-11.129,0.0,0.0448,0.044,0.443,0.239,0.4970,128.017
1,d50046ceb4db11dc162649a0da27733ff5194b47,SOAAAGQ12A8C1420C8,Orgelblut,1,Dolores,Bohren & Der Club Of Gore,2008,"dark jazz, drone",9.0,-21.904,0.0,0.0446,0.819,0.890,0.117,0.0659,129.709
2,1e93ab57208a575ad87872ff3a25c1743632d0a1,SOAAAGQ12A8C1420C8,Orgelblut,1,Dolores,Bohren & Der Club Of Gore,2008,"dark jazz, drone",9.0,-21.904,0.0,0.0446,0.819,0.890,0.117,0.0659,129.709
3,a0846981ab7c3d06cf1d966e7109774e306eb61a,SOAAAGQ12A8C1420C8,Orgelblut,1,Dolores,Bohren & Der Club Of Gore,2008,"dark jazz, drone",9.0,-21.904,0.0,0.0446,0.819,0.890,0.117,0.0659,129.709
4,6e8d4e7d986e077d334da41638e63b9030710141,SOAAAGQ12A8C1420C8,Orgelblut,1,Dolores,Bohren & Der Club Of Gore,2008,"dark jazz, drone",9.0,-21.904,0.0,0.0446,0.819,0.890,0.117,0.0659,129.709
